# Merge

Branches merge is opertion that allows you to get changes from intependend branches in one common commit. The `git merge` command is crucial for this page.

To merge branches you need to `git checkout` to brunch to which we merge all the changes, and execute command `git merge <branch to be merged>`.

## To ancestor{#sec-to_ancestor}

The simplest case is when you create a branch, make a few commits, and want to add those changes from an ancestor branch that hasn't changed.

Simply put, you just tell the parent branch to just start referencing the last commit of the branch you want to merge into the parent branch.

In the next cell:

- Created repository where there is `basic commit`;
- From `basic commit` branch `example_branch` was created;
- `branch_commit` was commited to `example_branch`, first `git log` display exactly this state of the repository;
- Then `example_branch` was merged to `master` - on `git log` it just desplayed as `master` moved to same commit as `example_branch`.

In [1]:
%%bash
mkdir merge_example
cd merge_example
git init &> /dev/null

echo "content" > test_file
git add --all
git commit -m "basic commit" &> /dev/null

echo
echo "=====creating branch====="
git checkout -b example_branch 2>&1
echo "content2" > test_file
git commit -am "branch commit" &> /dev/null
git log --decorate --graph

echo
echo "=====merging to master====="
git checkout master 2>&1

echo "-----git merge-----"
git merge example_branch
echo "-----git log-----"
git log --decorate --graph

cd ..
rm -r merge_example


=====creating branch=====
Switched to a new branch 'example_branch'
* commit bbd3463120a7fc513429a75ca2ebfff325571f84 (HEAD -> example_branch)
| Author: Dranikf <kobfedsur@gmail.com>
| Date:   Sun Sep 10 16:20:47 2023 +0300
| 
|     branch commit
| 
* commit c42d7deed4d71b0f5c9036816fe65e7295a5e227 (master)
  Author: Dranikf <kobfedsur@gmail.com>
  Date:   Sun Sep 10 16:20:47 2023 +0300
  
      basic commit

=====merging to master=====
Switched to branch 'master'
-----git merge-----
Updating c42d7de..bbd3463
Fast-forward
 test_file | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
-----git log-----
* commit bbd3463120a7fc513429a75ca2ebfff325571f84 (HEAD -> master, example_branch)
| Author: Dranikf <kobfedsur@gmail.com>
| Date:   Sun Sep 10 16:20:47 2023 +0300
| 
|     branch commit
| 
* commit c42d7deed4d71b0f5c9036816fe65e7295a5e227
  Author: Dranikf <kobfedsur@gmail.com>
  Date:   Sun Sep 10 16:20:47 2023 +0300
  
      basic commit


## Basic case

Here I'll play with the case when you have to branches but one doesn't refer to any ancestor of other (like it was in [To ancestor](#sec-to_ancestor)) section, so we have two separate branches.

**Note** in this example I don't pay any attention ot conflicts, the example is made in such a way as to avoid conflicts.

In the example:

- I created a repository where there are two branches that have independent commits in them, this is shown in the first `git log` output;
- Then I merge `example_branch` into `master`;
- In the final `git log` output:
    - There is a new commit where the two branches merge;
    - The message of the commit is `Merge branch 'example_branch'` - automatically generated by git, in practice you'll be able to set the message you want.  

In [32]:
%%bash
mkdir merge_example
cd merge_example
git init &> /dev/null

echo "content" > file
git add --all
git commit -m "basic commit" &> /dev/null

git checkout -b example_branch &> /dev/null
echo "content" > branch_file
git add --all
git commit -m "branch commit" &> /dev/null

git checkout master &> /dev/null
echo "master content" > file
git commit -am "master commit" &> /dev/null

echo
echo "=====log====="
git log --decorate --graph --all

echo
echo "=====merge====="
git merge example_branch

echo
echo "=====log====="
git log --decorate --graph --all

cd ..
rm -r merge_example


=====log=====
* commit 6ea69b66f6c2a49ea60a3a6b5b3de66181274234 (example_branch)
| Author: Fedor Kobak <kobfedsur@gmail.com>
| Date:   Sun Sep 10 13:00:54 2023 +0300
| 
|     branch commit
|   
| * commit 8b9f875b80b9d44020dedf40a39b1894f4a406d6 (HEAD -> master)
|/  Author: Fedor Kobak <kobfedsur@gmail.com>
|   Date:   Sun Sep 10 13:00:54 2023 +0300
|   
|       master commit
| 
* commit 15149be74a6d219dac74624c94bd7aeda0691e9d
  Author: Fedor Kobak <kobfedsur@gmail.com>
  Date:   Sun Sep 10 13:00:54 2023 +0300
  
      basic commit

=====merge=====
Merge made by the 'ort' strategy.
 branch_file | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 branch_file

=====log=====
*   commit 96f925a2adbada606ba4678c71460cf697fd8590 (HEAD -> master)
|\  Merge: 8b9f875 6ea69b6
| | Author: Fedor Kobak <kobfedsur@gmail.com>
| | Date:   Sun Sep 10 13:00:54 2023 +0300
| | 
| |     Merge branch 'example_branch'
| | 
| * commit 6ea69b66f6c2a49ea60a3a6b5b3de66181274234 (example_branch)
| | Autho